In [1]:
import os
import warnings
warnings.filterwarnings(action='ignore')

if os.getcwd() == '/content':
    data_path = '/content/drive/MyDrive/Colab Notebooks/datasets'
else:
    data_path = r'G:\내 드라이브\Colab Notebooks\datasets'

# MNIST


## idea

In [2]:
import torch
import torch.nn as nn

In [3]:
inputs = torch.Tensor(1, 1, 28, 28)
print(inputs.shape)

torch.Size([1, 1, 28, 28])


In [4]:
conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, 
                  stride=1, padding=1)
conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,
                  stride=1, padding=1)
pool = nn.MaxPool2d(kernel_size=2)

print(conv1, conv2, pool)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [5]:
print(inputs.shape)
out = conv1(inputs)
print(out.shape) # out.shape[1] == out_channel

torch.Size([1, 1, 28, 28])
torch.Size([1, 32, 28, 28])


In [6]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [7]:
out = conv2(out)
print(out.shape) # out.shape[1] == out_channel

torch.Size([1, 64, 14, 14])


In [8]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [9]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [10]:
fc = nn.Linear(in_features=3136, out_features=10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## CNN

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)

if device == 'cuda':
    torch.cuda.manual_seed_all(42)

In [13]:
learning_rate = 0.001
traning_epochs = 15
batch_size = 100

In [14]:
mnist_train = datasets.MNIST(root=data_path, train=True, download=True,
                             transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root=data_path, train=False, download=True,
                            transform=transforms.ToTensor())

data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size,
                         shuffle=True, drop_last=True)

In [15]:
temp, _ = next(iter(data_loader))
temp.shape

torch.Size([100, 1, 28, 28])

In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # input shape = (n, 1, 28, 28)
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, 
                      kernel_size=3, stride=1, padding=1), # (n, 32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # (n, 32, 14, 14)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64,
                      kernel_size=3, stride=1, padding=1), # (n, 64, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # (n, 64, 7, 7)
        )
        
        self.fc = nn.Linear(in_features=64*7*7, out_features=10, bias=True)
        nn.init.xavier_uniform_(self.fc.weight) # fc layer weight init
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(dim=0), -1) # flatten
        out = self.fc(out)
        return out

In [17]:
total_batch = len(data_loader)
print(f'total batch : {total_batch}')

model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(traning_epochs):
    
    avg_loss = 0
    
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        pred = model(x)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
    
    print(f'[epoch {epoch+1:>2d}/{traning_epochs}] loss : {avg_loss:>6f}')

total batch : 600
[epoch  1/15] loss : 0.231843
[epoch  2/15] loss : 0.061681
[epoch  3/15] loss : 0.045757
[epoch  4/15] loss : 0.036138
[epoch  5/15] loss : 0.030492
[epoch  6/15] loss : 0.024981
[epoch  7/15] loss : 0.021220
[epoch  8/15] loss : 0.017562
[epoch  9/15] loss : 0.015908
[epoch 10/15] loss : 0.012663
[epoch 11/15] loss : 0.011111
[epoch 12/15] loss : 0.008808
[epoch 13/15] loss : 0.008896
[epoch 14/15] loss : 0.007095
[epoch 15/15] loss : 0.005202


In [18]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = model(x_test)
    correct = torch.argmax(pred, 1) == y_test
    acc = correct.float().mean()
    
    print(f'accuracy : {acc.item()}')

accuracy : 0.9854999780654907


# deep CNN MNIST


In [19]:
import torch
from torch import nn
from torchvision import datasets, transforms

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)

if device == 'cuda':
    torch.cuda.manual_seed_all(42)

In [21]:
learning_rate = 0.001
traning_epochs = 15
batch_size = 100

In [22]:
mnist_train = datasets.MNIST(root=data_path, train=True, download=True,
                             transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root=data_path, train=False, download=True,
                            transform=transforms.ToTensor())

data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size,
                         shuffle=True, drop_last=True)

In [23]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.keep_prob = 0.5
        
        self.layer1 = nn.Sequential(
            # input shape = (n, 1, 28, 28)
            nn.Conv2d(in_channels=1, out_channels=32, 
                      kernel_size=3, stride=1, padding=1),
            # (n, 32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            # (n, 32, 14, 14)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, 
                      kernel_size=3, stride=1, padding=1),
            # (n, 64, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            # (n, 64, 7, 7)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, 
                      kernel_size=3, stride=1, padding=1),
            # (n, 128, 7, 7)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
            # (n, 128, 4, 4)
        )
        
        self.fc1 = nn.Linear(in_features=4*4*128, out_features=625, bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p=1-self.keep_prob)
        )
        
        self.fc2 = nn.Linear(in_features=625, out_features=10, bias=True)
        nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(dim=0), -1) # flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [24]:
total_batch = len(data_loader)
print(f'total batch : {total_batch}')

model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(traning_epochs):
    
    avg_loss = 0
    
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        pred = model(x)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
    
    print(f'[epoch {epoch+1:>2d}/{traning_epochs}] loss : {avg_loss:>6f}')

total batch : 600
[epoch  1/15] loss : 0.202598
[epoch  2/15] loss : 0.053756
[epoch  3/15] loss : 0.036657
[epoch  4/15] loss : 0.029192
[epoch  5/15] loss : 0.023687
[epoch  6/15] loss : 0.019849
[epoch  7/15] loss : 0.017206
[epoch  8/15] loss : 0.014700
[epoch  9/15] loss : 0.011014
[epoch 10/15] loss : 0.010909
[epoch 11/15] loss : 0.011727
[epoch 12/15] loss : 0.008070
[epoch 13/15] loss : 0.008460
[epoch 14/15] loss : 0.007494
[epoch 15/15] loss : 0.007788


In [25]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = model(x_test)
    correct = torch.argmax(pred, 1) == y_test
    acc = correct.float().mean()
    
    print(f'accuracy : {acc.item()}')

accuracy : 0.9297999739646912
